### Подготовка датасета по показателю объем убоя лошади

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_horse = df[df['Показатель'].isin(['Лошади', 'Температура', 'Осадки', 'Поголовье: лошади'])]
df_horse.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
181,Температура,ГАСТАНА,-13.074194,-10.628571,-5.941935,5.976667,15.767742,21.390000,20.803226,18.670968,12.386667,4.083871,-6.276667,-5.383871,-14.477419,-7.551724,-0.567742,9.860000,14.216129,18.296667,19.796774,20.119355,15.406667,0.516129,-10.513333,-9.983871,-10.838710,-13.428571,-6.283871,7.106667,15.487097,22.136667,20.945161,21.254839,13.230000,4.129032,-0.883333,-10.535484,-19.135484,-13.639286,-7.200000,5.000000,10.483871,18.486667,21.435484,18.141935,12.013333,5.219355,-6.606667,-14.354839,-12.848387,-13.010714,-2.341935,6.463333,14.170968,17.493333,23.293548,20.525806,12.096667,8.074194,-7.923333,-7.732258,-8.954839,-6.379310,-3.106452,10.830000,18.909677,19.243333,22.077419,19.948387,12.106667,4.696774,-6.203333,-15.709677,-16.022581,-12.060714,-7.238710,6.073333,18.741935,18.523333,21.687097,20.935484,10.670000,4.048387,-5.323333,-8.893548,-11.261290,-9.150000,-7.248387,9.770000,16.487097,20.363333,21.854839,18.280645,15.503333,6.129032,-5.980000,-15.283871,-14.025806,-12.310714,-1.409677,5.810000,15.780645,21.240000,24.941935,20.629032,13.333333,7.487097,2.213333,-8.925806,-11.941935,-13.475862,-4.432258,9.120000,12.300000,21.836667,21.767742,18.545161,11.260000,5.541935,-3.520000,-8.758065
44,Лошади,АЛМАТИНСКАЯ ОБЛАСТЬ,1558.770000,1160.780000,3029.590000,796.150000,827.190000,3031.950000,693.890000,2782.580000,3034.640000,1688.080000,3297.070000,5715.090000,1526.990000,1142.720000,3169.530000,756.610000,776.860000,3246.460000,638.600000,2738.230000,3044.830000,1788.240000,3824.910000,6649.320000,1551.390000,1182.050000,3542.150000,868.630000,745.370000,3700.330000,619.050000,2624.530000,3427.190000,2138.010000,4212.380000,7386.400000,1555.300000,1276.050000,3753.180000,962.020000,843.730000,4358.600000,662.560000,2725.500000,3583.190000,2311.690000,4348.210000,7007.660000,1630.560000,1293.390000,3798.150000,1032.720000,829.050000,4499.690000,583.360000,3016.310000,3744.690000,2398.800000,4395.530000,6782.160000,1674.810000,1445.630000,3893.430000,1025.390000,908.780000,4801.620000,596.960000,3185.520000,4015.880000,2518.150000,4628.600000,7087.710000,1744.690000,1380.710000,4270.280000,1184.610000,1424.090000,5024.400000,671.930000,3400.700000,4265.720000,2673.090000,4783.700000,7800.320000,1786.850000,1404.460000,4578.360000,1230.380000,1330.570000,2215.000000,483.960000,1929.320000,2779.500000,1766.170000,2917.000000,5183.850000,569.490000,490.480000,2928.410000,1034.750000,732.920000,1959.690000,349.430000,1585.480000,2379.830000,1534.230000,2557.010000,4352.660000,407.000000,752.270000,2940.250000,983.900000,746.070000,2097.700000,364.170000,1573.540000,2522.070000,1535.100000,2563.150000,5168.280000
318,Осадки,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,23.700000,7.400000,3.100000,35.800000,25.400000,28.000000,23.700000,9.500000,32.400000,26.100000,77.400000,31.800000,44.600000,32.400000,21.100000,39.900000,69.400000,16.600000,42.700000,2.800000,50.300000,24.800000,26.600000,39.700000,8.000000,20.400000,21.200000,32.100000,13.700000,49.900000,17.300000,6.00000

In [5]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_horse.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_horse = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_horse.groupby("Регион").size().reset_index(name="Количество строк"))
df_horse

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Лошади,Осадки,Поголовье: лошади,Температура
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,1292.32,9.8,129282.0,-12.490323
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,704.43,9.8,139079.0,-10.192857
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,721.88,8.3,142545.0,-5.870968
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,742.10,8.8,154110.0,4.490000
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,1041.54,42.8,166680.0,14.574194
...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,1955.90,0.0,474702.0,27.874194
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,3893.88,0.5,470317.0,20.766667
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,4357.98,13.6,464893.0,13.200000
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,5998.64,12.1,456239.0,6.500000


In [7]:
df_horse = df_horse[df_horse["Регион"] != 'РЕСПУБЛИКА КАЗАХСТАН']
df_horse = df_horse.drop(columns=['Осадки', 'Поголовье: лошади', 'Температура'])
df_horse.sample(10)

Показатель,Регион,Период,Лошади
448,АТЫРАУСКАЯ ОБЛАСТЬ,2022-05,859.73
792,ГАСТАНА,2021-01,4.72
1116,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2021-06,1736.70
2002,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2017-07,2178.98
729,ГАСТАНА,2015-10,6.02
722,ГАСТАНА,2015-03,5.67
178,АКТЮБИНСКАЯ ОБЛАСТЬ,2019-11,1991.46
392,АТЫРАУСКАЯ ОБЛАСТЬ,2017-09,638.22
1720,ОБЛАСТЬ ҰЛЫТАУ,2024-01,2131.20
810,ГАСТАНА,2022-07,1.10


In [8]:
df_horse.to_excel("Датасет по лошадям.xlsx", index=False)